# 1. Simple Dataset reading pipeline

In [ ]:
import torch
import torch.nn as nn
import torchvision, torchinfo, torchmetrics
import torchvision
from sklearn.model_selection import train_test_split
import os, glob, zipfile
from tqdm import tqdm
from PIL import Image

def DOWNLOAD_DATASETS():
    zip_files = ['test.zip', 'train.zip']

    for zip_file in zip_files:
        with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/{}".format(zip_file),"r") as z:
            z.extractall(".")
            print("{} unzipped".format(zip_file))

    train_file_names_list = glob.glob(os.path.join("../working/train",'*.jpg'))
    test_file_names_list  = glob.glob(os.path.join("../working/test", '*.jpg'))

    train_list, val_list  = train_test_split(train_file_names_list, test_size=0.2)

    transformation_list =  torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)), # IMPORTANT SIZE OF IMAGE (224, 224) or (256,256)
            torchvision.transforms.ToTensor(),
        ])

    class Custom_Dataset(torch.utils.data.Dataset):
        def __init__(self,file_list,transformation_list = None):
            self.file_list = file_list
            self.transform = transformation_list

        def __len__(self):
            self.filelength = len(self.file_list)
            return self.filelength

        def __getitem__(self,idx):
            img_path = self.file_list[idx]
            img = Image.open(img_path)
            img_transformed = self.transform(img)

            label = img_path.split('/')[-1].split('.')[0]
            if label == 'dog':
                label=1
            elif label == 'cat':
                label=0

            return img_transformed,label

    train_dataset = Custom_Dataset(train_list, transformation_list)
    val_dataset   = Custom_Dataset(val_list  , transformation_list)

    train_loader  = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=32, shuffle=True )
    val_loader    = torch.utils.data.DataLoader(dataset = val_dataset, batch_size=32, shuffle=True)
    
    return train_dataset, val_dataset, train_loader, val_loader


training_dataset, validation_dataset, training_dataloader, validation_dataloader = DOWNLOAD_DATASETS();
assert next(iter(training_dataloader)) is not None
assert next(iter(validation_dataloader)) is not None

# 2. Simple Model Training Pipeline

In [ ]:
lr      = 0.001 # learning_rate
epochs  = 10 # How much to train a model
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def TRAIN_MODEL(model, training_dataloader, validation_dataloader):
    
    model.train(mode=True)
    OPTIMIZER = torch.optim.SGD ( params= model.parameters(), lr= lr ) # Using torch.optimizer algorithm
    metric    = torchmetrics.Accuracy(task="multiclass", num_classes= 2 ).to(device)
    
    for epoch_no in range(epochs):
        for batch_no, (image_tensors, labels) in enumerate(progress_bar := tqdm(training_dataloader)):
            
            x_actual, y_actual = image_tensors.to(device), labels.to(device)
            
            y_predicted_LOGITS = model.forward               (x_actual)
            y_predicted_probs  = nn.functional.softmax       (y_predicted_LOGITS, dim= 1)
            loss               = nn.functional.cross_entropy (y_predicted_LOGITS, y_actual.to(torch.int64))
            
            OPTIMIZER.zero_grad()
            loss.backward()
            # dError_dParameters    = torch.autograd.grad( outputs = ERROR_FUNC( y_predicted, y_actual ), inputs = model.parameters())
            # Parameters of layer 1 are not dependent on any other parameters
            # Parameters of layer 2 are dependent on layer 1 parameters
            # Parameters of layer 3 are dependent on layer 2 parameters which are dependent on layer 1 parameters
            # Finding complicated rate of change of such nested parameters is done automatically when we do loss.backward()
            OPTIMIZER.step()
            """
            for (name, weight), gradient in zip(model.named_parameters(), dError_dWeights):
                weight = weight - gradient * LEARNING_RATE
                print(f"Parameters of layer: {name} have these many {torch.count_nonzero(gradient)} updates out of {torch.count(gradient)})
            """

            loss_batch      = loss.item()
            accuracy_batch  = metric(y_predicted_LOGITS, y_actual)
            train_acc_epoch = metric.compute() # calculates average accuracy across epoch automatically

            metrics_per_batch = {
                "loss_batch": loss_batch,
                "accuracy_running_average": train_acc_epoch,
            }
            progress_bar.set_description(f'batch_no = {batch_no},\t loss_batch = {loss_batch:0.4f},\t accuracy_avg = {train_acc_epoch:0.4f}')
            
        metric.reset()
        loss_validation, accuracy_validation = EVALUATE_MODEL(model, validation_dataloader)
        print(f'epoch_no = {epoch_no}, training_loss = {loss_batch:0.4f}, validation_loss = {loss_validation:0.4f},\t training_accuracy = {accuracy_batch:0.4f}, validation_accuracy = {accuracy_validation:0.4f}')
    
    model.train(mode=False)


def EVALUATE_MODEL(model, validation_dataloader):
    # EVALUATE MODEL AT END OF EVERY EPOCH
    model.eval()
    metric = torchmetrics.Accuracy(task="multiclass", num_classes= 2 ).to(device)
    with torch.no_grad():
        for batch_no, (image_tensors, labels) in enumerate(validation_dataloader):
            x_actual, y_actual = image_tensors.to(device), labels.to(device)

            y_predicted_LOGITS = model.forward                 (x_actual)
            loss               = nn.functional.cross_entropy   (y_predicted_LOGITS, y_actual.to(torch.int64)).item()
            accuracy_batch     = metric                        (y_predicted_LOGITS, y_actual).item()

        testing_accuracy_avg = metric.compute().item()
    return loss, testing_accuracy_avg

# 3. Simple Transfer Learning

In [24]:
import timm

test_image = torch.randn(2, 3, 224, 224)

pre_trained_model = timm.create_model('vgg16'                , pretrained=True, )

print(pre_trained_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [25]:
pre_trained_model.head

ClassifierHead(
  (global_pool): SelectAdaptivePool2d(pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=4096, out_features=1000, bias=True)
  (flatten): Identity()
)

In [26]:
pre_trained_model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [27]:
pre_trained_model.pre_logits

ConvMlp(
  (fc1): Conv2d(512, 4096, kernel_size=(7, 7), stride=(1, 1))
  (act1): ReLU(inplace=True)
  (drop): Dropout(p=0.0, inplace=False)
  (fc2): Conv2d(4096, 4096, kernel_size=(1, 1), stride=(1, 1))
  (act2): ReLU(inplace=True)
)

In [ ]:
transfer_learning_model = nn.Sequential(
    pre_trained_model,
    nn.Flatten(start_dim=1),
    nn.Dense(in_features = 512*7*7, out_features = 2),
)

In [ ]:
torchinfo.summary(transfer_learning_model, input_size= (1,3,224,224) )

## Standard transfer learning models

In [ ]:
m = timm.create_model('resnet50'             , pretrained=True)
m = timm.create_model('densenet121'          , pretrained=True)
m = timm.create_model('efficientnet_b0'      , pretrained=True)
m = timm.create_model('inception_v4'         , pretrained=True)
m = timm.create_model('mobilenetv2_100'      , pretrained=True)
m = timm.create_model('inception_v4'         , pretrained=True)
m = timm.create_model('inception_v4'         , pretrained=True)

In [13]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet',       pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True) # Alexnet level accuracy with 50x fewer parameters
model = torch.hub.load("coderx7/simplenet_pytorch:v1.0.0", "simplenetv1_5m_m1", pretrained=True)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 145MB/s]  
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/conda/lib/pyt

saving in checkpoint_path:tmp/simplenetv1_5m_m1-36c4ca4d.pth
